In [2]:
import os
import numpy as np
import math

from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch

########### Вывод изображений и сохранение ################################
import matplotlib.pyplot as plt
from IPython import display
from torchvision.utils import save_image, make_grid

In [7]:
use_cuda = torch.cuda.is_available()
# device = torch.device("cpu")
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


Как было сказано ранее, GAN обучается воспроизводить реальные данные. Поэтому Вам предлагается обучить генератор создавать точки, которые будут лежать на графике функции 𝑦=𝑠𝑖𝑛(𝑥)𝑥−𝑥10. При выполненинии данного задания структура GAN остается той же, но Вам нужно:

    Сгенерировать настоящие данные
    Изменить архитектуру дискриминатора и генератора
    Без графиков домашку не принимаю


In [12]:


train_data_length = 1024

train_data = torch.zeros((train_data_length, 2))
xx = np.linspace(-10, 10, train_data_length)
train_data[:, 0] = torch.tensor(xx)
train_data[:, 1] = torch.tensor(func_y(xx))
train_labels = torch.ones(train_data_length)
train_set = [
    (train_data[i], train_labels[i]) for i in range(train_data_length)]



In [28]:
def func_y(x):
    return np.sin(x)/x - x/10

train_data_x = np.linspace(1,100,2048)
train_data_y = func_y(train_data_x)
train_data = list(zip(train_data_x, train_data_y))

In [9]:
n_epochs = 200
batch_size = 32
lr = 0.0002

In [29]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

In [30]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        
        def block(in_feat, out_feat):
            layers = [nn.Linear(in_feat, out_feat)]
            layers.append(nn.ReLU(0.2))
            return layers

        self.model = nn.Sequential(
            *block(2, 128),
            *block(128, 256),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape)
        return img


In [6]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()

        def block(in_feat, out_feat):
            layers = [nn.Linear(in_feat, out_feat)]
            layers.append(nn.ReLU(0.2))
            return layers
        
        self.model = nn.Sequential(
            CustomLinearLayer(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            CustomLinearLayer(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            CustomLinearLayer(256, 1),
            nn.Sigmoid(),
        )
  

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)

        return validity

In [15]:
generator = Generator().to(device)
discriminator = Discriminator().to(device)


optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

adversarial_loss = torch.nn.BCELoss()

NameError: name 'CustomLinearLayer' is not defined

In [4]:
a = [2,3]

In [5]:
np.prod(a)

6